In [1]:
import os
if not os.path.isfile("estrutura_ud.py"):
    os.system("wget https://raw.githubusercontent.com/alvelvis/ACDC-UD/master/estrutura_ud.py")
if not os.path.isfile("tokenization.py"):
    os.system("wget https://raw.githubusercontent.com/alvelvis/ACDC-UD/master/tokenization.py")
import estrutura_ud
import tokenization
import pprint
import random

file = "Petroles_3_golden.conllu"
release_file = "PetroGold.conllu"

def change_col(col, token, value):
    col = token.__dict__[col].split("|")
    col = [x for x in col if not x.startswith(value.split("=")[0]) and x != "_"] + [value]
    col = "|".join(sorted(col))
    return col

def most_common(lst):
    return max(set(lst), key=lst.count)

def count_tokens(corpus, i=0):
    for sentence in corpus.sentences.values():
        i += len([x for x in sentence.tokens if not '-' in x.id])
    return i

def count_words(corpus, i=0):
    for sentence in corpus.sentences.values():
        i += len([x for x in sentence.tokens if not '-' in x.id and x.upos != "PUNCT"])
    return i

def count_documents(corpus):
    return len(set([x.rsplit("-", 1)[0] for x in corpus.sentences]))

def count_entities(corpus, i=0):
    for sentence in corpus.sentences.values():
        i += len([x for x in sentence.tokens if 'B=' in x.deps])
    return i

def count_all(label, corpus, stats=""):
    if not stats:
        stats = {'sentences': {}, 'tokens': {}, 'entities': {}, 'documents': {}, 'words': {}}
    stats['sentences'][label] = len(corpus.sentences)
    stats['tokens'][label] = count_tokens(corpus)
    stats['documents'][label] = count_documents(corpus)
    stats['entities'][label] = count_entities(corpus)
    stats['words'][label] = count_words(corpus)
    return stats

In [ ]:
# COSMO sem root e ciclo
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

for sentence in corpus.sentences.values():
    if not any(x.deprel == "root" for x in sentence.tokens):
        sentence.tokens[0].misc = "SEM ROOT"
    elif sentence.to_str().count("\troot\t") > 1:
        for token in sentence.tokens:
            if token.deprel == "root":
                token.misc = "MAIS DE 1 ROOT"
    else:
        for token in sentence.tokens:
            if (
                token.dephead != "0" and (
                    token.dephead == token.head_token.dephead or 
                    token.dephead == token.id or 
                    token.id == token.head_token.dephead or 
                    (token.head_token.dephead != "0" and token.head_token.head_token.dephead == token.id) or
                    (token.head_token.dephead != "0" and token.head_token.head_token.dephead != "0" and token.head_token.head_token.head_token.dephead == token.id)
                )):
                token.misc = "CICLO"
            if token.dephead == "0" and token.deprel != "root":
                token.misc = "root"
            if token.dephead != "0" and token.deprel == "root":
                token.misc = "root"

file_name = file.rsplit("/", 1)[1] if '/' in file else file
corpus.save("regras.conllu")
os.system("python3 ../../conllu-merge-resolver/cosmo.py {} regras.conllu '.*'".format(file))
os.system("cp {} antes-{}".format(file, file_name))
os.system("rm regras-correcao.zip; zip regras-correcao.zip antes-{} regras.conllu".format(file_name))
os.remove("antes-{}".format(file_name))
os.remove("regras.conllu")

In [ ]:
# correções de formato: ordem alfabética das feats, =, e deprels que não podem ter filho
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

for sentence in corpus.sentences.values():
    for token in sentence.tokens:
        if '</' in token.to_str():
            token.misc = "</"
        if ' ' in token.to_str():
            token.misc = "espaço em branco"
        if not token.feats.strip():
            token.feats = "_"
        if token.feats != "_":
            try:
                parts = [x.split('=') for x in token.feats.split("|")]
                parts = dict(parts)
            except ValueError:
                token.misc = "feats com erro"
            token.feats = "|".join(sorted(token.feats.split("|"), key=lambda x: x.lower()))
    
    cant_have_children = "flat:name compound punct fixed aux cop case mark punct".split()
    for token in sentence.tokens:
        if token.deprel not in "fixed compound".split() and token.head_token.deprel in cant_have_children:
            token.dephead = token.head_token.dephead if token.head_token.head_token.deprel not in cant_have_children else token.head_token.head_token.dephead

file_name = file.rsplit("/", 1)[1] if '/' in file else file
corpus.save("regras.conllu")
os.system("python3 ../../conllu-merge-resolver/cosmo.py {} regras.conllu '.*'".format(file))
os.system("cp {} antes-{}".format(file, file_name))
os.system("rm regras-correcao.zip; zip regras-correcao.zip antes-{} regras.conllu".format(file_name))
os.remove("antes-{}".format(file_name))
os.remove("regras.conllu")

In [ ]:
# TEXT IGUAIS
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

texts = {}
for sentid, sentence in corpus.sentences.items():
    text = sentence.text
    if not text in texts:
        texts[text] = []
    texts[text].append(sentid)

with open("frases-repetidas.txt", "w+") as f:
    for text in texts:
        if len(texts[text]) >= 2:
            f.write("{}\n{}\n\n".format(text, "|".join(texts[text])))

In [ ]:
# text contrações
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

changed_sentences = []
for sentence in corpus.sentences.values():
    if not sentence.text.endswith(sentence.tokens[-1].word):
        contractions = [[int(x.id.split("-")[0]), int(x.id.split("-")[1]), x.word] for x in sentence.tokens if '-' in x.id]
        new_text = [x.word for x in sentence.tokens if not '-' in x.id]
        for contraction in reversed(contractions):
            start = contraction[0]
            end = contraction[1]
            word = contraction[2]
            for i in reversed(range(start-1, end)):
                new_text.pop(i)
            new_text.insert(start-1, word)
        new_text = " ".join(new_text)
        sentence.metadados['text'] = new_text
        changed_sentences.append(sentence.sent_id)

print("|".join(changed_sentences))

corpus.save("regras.conllu")

In [ ]:
# eliminando pontuação duplicada
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

changed_sentences = []
for sentid, sentence in corpus.sentences.items():
    if sentence.tokens[-1].word == "." and sentence.tokens[-2].word == ".":
        sentence = tokenization.addToken(corpus, sentid, 'rm', sentence.tokens[-1].id).sentences[sentid]
        sentence.tokens[-1].dephead = [x for x in sentence.tokens if x.deprel == "root"][0].id
        changed_sentences.append(sentence.sent_id)
    if sentence.tokens[-1].word == "..":
        sentence.tokens[-1].word = "."
        sentence.tokens[-1].lemma = "."
        sentence.tokens[-1].dephead = [x for x in sentence.tokens if x.deprel == "root"][0].id
        changed_sentences.append(sentence.sent_id)
    if sentence.text.endswith(".."):
        sentence.metadados['text'] = sentence.text[:-1]
        changed_sentences.append(sentence.sent_id)

print("|".join(changed_sentences))

corpus.save("regras.conllu")

In [ ]:
# tokens com ..
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

for sentid, sentence in corpus.sentences.items():
    if sentence.tokens[-1].word.endswith("..") and (sentence.tokens[-1].upos == "PUNCT" or sentence.tokens[-1].deprel == "punct"):
        sentence.tokens[-1].upos = "EPA"
        sentence.tokens[-1].misc = sentence.tokens[-1].misc.replace("tokenização", "").replace("||", "|")
        if not sentence.tokens[-1].misc:
            sentence.tokens[-1].misc = "_"

corpus.save("regras.conllu")
os.system("python3 ../../conllu-merge-resolver/cosmo.py {} regras.conllu '.*'".format(file))
os.system("cp {} antes-{}".format(file, file_name))
os.system("rm regras-correcao.zip; zip regras-correcao.zip antes-{} regras.conllu".format(file_name))
os.remove("antes-{}".format(file_name))
os.remove("regras.conllu")

In [ ]:
# tokens com .. parte 2
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

changed_sentences = []
for sentid, sentence in corpus.sentences.items():
    if sentence.tokens[-1].word.endswith(".."):
        sentence = tokenization.addToken(corpus, sentid, 'add', str(int(sentence.tokens[-1].id)+1), new_tokens=[]).sentences[sentid]
        sentence.tokens[-1].dephead = [x for x in sentence.tokens if x.deprel == "root"][0].id
        sentence.tokens[-1].lemma = "."
        sentence.tokens[-1].word = "."
        sentence.tokens[-1].deprel = "punct"
        sentence.tokens[-1].upos = "PUNCT"
        sentence.tokens[-2].word = sentence.tokens[-2].word[:-2]
        sentence.tokens[-2].lemma = sentence.tokens[-2].lemma[:-2]
        changed_sentences.append(sentence.sent_id)

print("|".join(changed_sentences))

corpus.save("regras.conllu")

In [ ]:
# eliminando frases
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

old_sentences = set(corpus.sentences.keys())
corpus.sentences = {x: corpus.sentences[x] for x in corpus.sentences if not x.startswith("258-")}
corpus.sentences = {x: corpus.sentences[x] for x in corpus.sentences if not any(y in corpus.sentences[x].to_str() for y in "sentenciação".split())}
corpus.sentences = {x: corpus.sentences[x] for x in corpus.sentences if not any(y in corpus.sentences[x].to_str() for y in "tokenização".split())}
corpus.sentences = {x: corpus.sentences[x] for x in corpus.sentences if not any(y in corpus.sentences[x].to_str() for y in "eliminar".split())}
new_sentences = set(corpus.sentences.keys())

if new_sentences != old_sentences:
    print(len(new_sentences))
    print(len(old_sentences))

corpus.save("regras.conllu")

In [19]:
# list sentences removed
pre_commit = "48efd0a9d2092cd8f4dd17c0eda7bcc85522f02c"
pos_commit = "master"

os.system("git checkout {}".format(pre_commit))
corpus_pre = estrutura_ud.Corpus(recursivo=True)
corpus_pre.load(file)

os.system("git checkout {}".format(pos_commit))
corpus_pos = estrutura_ud.Corpus(recursivo=True)
corpus_pos.load(file)

stats = count_all("pré", corpus_pre)
stats = count_all("pós", corpus_pos, stats)

sent_ids = set(corpus_pre.sentences)
sent_ids_removed = sent_ids - set(corpus_pos.sentences)
print("|".join(sent_ids_removed))

pprint.pprint(stats)

error: Your local changes to the following files would be overwritten by checkout:
	Petroles_3/prepare_release.ipynb
Please commit your changes or stash them before you switch branches.
Aborting


KeyboardInterrupt: 

In [2]:
# remover sema, xpos, misc, nmod:appos (só para release)
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

for sentid, sentence in corpus.sentences.items():
    for token in sentence.tokens:
        token.deps = "_"
        token.xpos = "_"
        token.misc = "_"
        if token.deprel == "nmod:appos":
            token.deprel = "nmod"
        if token.deprel == "expl:impers":
            token.deprel = "expl"

corpus.save(release_file)

for sentid, sentence in corpus.sentences.items():
    for token in sentence.tokens:
        if token.deprel == "obl:arg":
            token.deprel = "obl"

corpus.save("experimento-obl-arg.conllu")

build: 8.406656980514526

In [ ]:
# VALIDATE.PY => erros de formato devem ser corrigidos
! ~/Documentos/Interrogat-rio/www/cgi-bin/tools/validate.py PetroGold.conllu --lang=en

In [3]:
# SEPARAR TRAIN E TEST
files = [release_file, "experimento-obl-arg.conllu"]

sentences = {}
get_sentences_for_partitions = {}

# comment this line if randomly generate partitions
get_sentences_for_partitions = {
    "dev": "PetroGold-v1/petrogold-dev.conllu",
    "train": "PetroGold-v1/petrogold-train.conllu",
    "test": "PetroGold-v1/petrogold-test.conllu"
    }

if get_sentences_for_partitions:
    for partition in get_sentences_for_partitions:
        corpus = estrutura_ud.Corpus()
        corpus.load(get_sentences_for_partitions[partition])
        sentences[partition] = set(corpus.sentences)        

for file in files:
    corpus = estrutura_ud.Corpus(recursivo=True)
    corpus.load(file)

    if not sentences:
        all_sentences = set(corpus.sentences)
        shuffled = list(corpus.sentences)
        random.shuffle(shuffled)
        sentences["test"] = set(shuffled[:round(0.05*len(all_sentences))])
        sentences["dev"] = set([x for x in shuffled if x not in sentences['test']][:round(0.05*len(all_sentences))])
        sentences["train"] = all_sentences - sentences["test"] - sentences["dev"]
    assert not sentences["train"].intersection(sentences["test"])
    assert not sentences["train"].intersection(sentences["dev"])
    assert not sentences["test"].intersection(sentences["dev"])

    train = []
    test = []
    dev = []
    for sentid, sentence in corpus.sentences.items():
        if sentid in sentences["test"]:
            test.append(sentence.to_str())
        if sentid in sentences["train"]:
            train.append(sentence.to_str())
        if sentid in sentences["dev"]:
            dev.append(sentence.to_str())

    corpus_train = estrutura_ud.Corpus(recursivo=True)
    corpus_test = estrutura_ud.Corpus(recursivo=True)
    corpus_dev = estrutura_ud.Corpus(recursivo=True)
    corpus_train.build("\n\n".join(train) + "\n\n")
    corpus_test.build("\n\n".join(test) + "\n\n")
    corpus_dev.build("\n\n".join(dev) + "\n\n")

    stats = count_all("train", corpus_train)
    stats = count_all("test", corpus_test, stats)
    stats = count_all("dev", corpus_dev, stats)
    pprint.pprint(stats)

    corpus_train.save("{}-train.conllu".format(file.rsplit(".", 1)[0]))
    corpus_test.save("{}-test.conllu".format(file.rsplit(".", 1)[0]))
    corpus_dev.save("{}-dev.conllu".format(file.rsplit(".", 1)[0]))

build: 0.5454018115997314build: 8.117834091186523build: 0.5099120140075684build: 9.190043687820435build: 7.901104927062988build: 8.214680671691895build: 8.56426191329956

{'documents': {'dev': 19, 'test': 19, 'train': 19},
 'entities': {'dev': 0, 'test': 0, 'train': 0},
 'sentences': {'dev': 447, 'test': 447, 'train': 8061},
 'tokens': {'dev': 12763, 'test': 12214, 'train': 225857},
 'words': {'dev': 11313, 'test': 10783, 'train': 199319}}


build: 9.615033626556396build: 8.165860652923584build: 8.307520866394043build: 8.608531475067139

{'documents': {'dev': 19, 'test': 19, 'train': 19},
 'entities': {'dev': 0, 'test': 0, 'train': 0},
 'sentences': {'dev': 447, 'test': 447, 'train': 8061},
 'tokens': {'dev': 12763, 'test': 12214, 'train': 225857},
 'words': {'dev': 11313, 'test': 10783, 'train': 199319}}


In [4]:
# TREINAR COM TRAIN E DEV
! ./udpipe-1.2.0 --train --tokenizer=none "PetroGold.udpipe" "PetroGold-train.conllu" "PetroGold-dev.conllu"
! ./udpipe-1.2.0 --train --tokenizer=none "experimento-obl-arg.udpipe" "experimento-obl-arg-train.conllu" "experimento-obl-arg-dev.conllu"
#! ./udpipe-1.2.0 --train --tokenizer=none "bosque-ud-2.8/bosque2.8.udpipe" "bosque-ud-2.8/pt_bosque-ud-train.conllu" "bosque-ud-2.8/pt_bosque-ud-dev.conllu"

Loading training data: done.
Training the UDPipe model.
Tagger model 1 columns: lemma use=1/provide=1, xpostag use=1/provide=1, feats use=1/provide=1
Creating morphological dictionary for tagger model 1.
Tagger model 1 dictionary options: max_form_analyses=0, custom dictionary_file=none
Tagger model 1 guesser options: suffix_rules=8, prefixes_max=4, prefix_min_count=10, enrich_dictionary=6
Tagger model 1 options: iterations=20, early_stopping=0, templates=tagger
Training tagger model 1.
Iteration 1: done, accuracy 92.46%
Iteration 2: done, accuracy 96.35%
Iteration 3: done, accuracy 97.26%
Iteration 4: done, accuracy 97.82%
Iteration 5: done, accuracy 98.14%
Iteration 6: done, accuracy 98.40%
Iteration 7: done, accuracy 98.56%
Iteration 8: done, accuracy 98.76%
Iteration 9: done, accuracy 98.87%
Iteration 10: done, accuracy 98.96%
Iteration 11: done, accuracy 99.05%
Iteration 12: done, accuracy 99.11%
Iteration 13: done, accuracy 99.17%
Iteration 14: done, accuracy 99.24%
Iteration 15:

In [5]:
# EVALUATE PETRO3
! python3 ~/Documentos/ACDC-UD/tokenizar_conllu.py "PetroGold-test.conllu" "PetroGold-test-tokenizado.txt"
! python3 ~/Documentos/ACDC-UD/udpipe_vertical.py "PetroGold.udpipe" "PetroGold-test-tokenizado.txt" "PetroGold-test-udpipe.conllu"
! python3 ~/Documentos/ACDC-UD/conll18_ud_eval.py -v "PetroGold-test.conllu" "PetroGold-test-udpipe.conllu"

OK!
Loading UDPipe model: done.
Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |    100.00 |    100.00 |    100.00 |
Sentences  |    100.00 |    100.00 |    100.00 |
Words      |    100.00 |    100.00 |    100.00 |
UPOS       |     98.33 |     98.33 |     98.33 |     98.33
XPOS       |    100.00 |    100.00 |    100.00 |    100.00
UFeats     |     97.72 |     97.72 |     97.72 |     97.72
AllTags    |     97.00 |     97.00 |     97.00 |     97.00
Lemmas     |     98.53 |     98.53 |     98.53 |     98.53
UAS        |     90.44 |     90.44 |     90.44 |     90.44
LAS        |     88.82 |     88.82 |     88.82 |     88.82
CLAS       |     83.57 |     83.23 |     83.40 |     83.23
MLAS       |     80.38 |     80.05 |     80.22 |     80.05
BLEX       |     81.94 |     81.60 |     81.77 |     81.60


In [6]:
# EVALUATE EXPERIMENTO OBL ARG
! python3 ~/Documentos/ACDC-UD/tokenizar_conllu.py "experimento-obl-arg-test.conllu" "experimento-obl-arg-test-tokenizado.txt"
! python3 ~/Documentos/ACDC-UD/udpipe_vertical.py "experimento-obl-arg.udpipe" "experimento-obl-arg-test-tokenizado.txt" "experimento-obl-arg-test-udpipe.conllu"
! python3 ~/Documentos/ACDC-UD/conll18_ud_eval.py -v "experimento-obl-arg-test.conllu" "experimento-obl-arg-test-udpipe.conllu"

OK!
Loading UDPipe model: done.
Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |    100.00 |    100.00 |    100.00 |
Sentences  |    100.00 |    100.00 |    100.00 |
Words      |    100.00 |    100.00 |    100.00 |
UPOS       |     98.33 |     98.33 |     98.33 |     98.33
XPOS       |    100.00 |    100.00 |    100.00 |    100.00
UFeats     |     97.72 |     97.72 |     97.72 |     97.72
AllTags    |     97.00 |     97.00 |     97.00 |     97.00
Lemmas     |     98.53 |     98.53 |     98.53 |     98.53
UAS        |     90.11 |     90.11 |     90.11 |     90.11
LAS        |     88.32 |     88.32 |     88.32 |     88.32
CLAS       |     82.69 |     82.31 |     82.50 |     82.31
MLAS       |     79.42 |     79.05 |     79.24 |     79.05
BLEX       |     81.02 |     80.65 |     80.84 |     80.65


In [7]:
# EVALUATE BOSQUE2.8
#! python3 ~/Documentos/ACDC-UD/tokenizar_conllu.py "bosque-ud-2.8/pt_bosque-ud-test.conllu" "bosque-ud-2.8/pt_bosque-ud-test-tokenizado.txt"
#! python3 ~/Documentos/ACDC-UD/udpipe_vertical.py "bosque-ud-2.8/bosque2.8.udpipe" "bosque-ud-2.8/pt_bosque-ud-test-tokenizado.txt" "bosque-ud-2.8/pt_bosque-ud-test-udpipe.conllu"
! python3 ~/Documentos/ACDC-UD/conll18_ud_eval.py -v "bosque-ud-2.8/pt_bosque-ud-test.conllu" "bosque-ud-2.8/pt_bosque-ud-test-udpipe.conllu"

Metric     | Precision |    Recall |  F1 Score | AligndAcc
-----------+-----------+-----------+-----------+-----------
Tokens     |    100.00 |    100.00 |    100.00 |
Sentences  |    100.00 |    100.00 |    100.00 |
Words      |    100.00 |    100.00 |    100.00 |
UPOS       |     96.52 |     96.52 |     96.52 |     96.52
XPOS       |    100.00 |    100.00 |    100.00 |    100.00
UFeats     |     95.12 |     95.12 |     95.12 |     95.12
AllTags    |     93.32 |     93.32 |     93.32 |     93.32
Lemmas     |     96.95 |     96.95 |     96.95 |     96.95
UAS        |     85.83 |     85.83 |     85.83 |     85.83
LAS        |     81.59 |     81.59 |     81.59 |     81.59
CLAS       |     73.80 |     73.53 |     73.67 |     73.53
MLAS       |     66.45 |     66.21 |     66.33 |     66.21
BLEX       |     70.45 |     70.19 |     70.32 |     70.19


In [17]:
# COUNT MODIFICATIONS
file = "Petroles_3_golden.conllu"
file_stanza = "Petroles_3_stanza.conllu"
file_udpipe = "Petroles_3_udpipe.conllu"
post_changes = "master"

os.system("git checkout {}".format(post_changes))
corpus_pos = estrutura_ud.Corpus(recursivo=True)
corpus_pos.load(file)

corpus_stanza = estrutura_ud.Corpus(recursivo=True)
corpus_stanza.load(file_stanza)
corpus_udpipe = estrutura_ud.Corpus(recursivo=True)
corpus_udpipe.load(file_udpipe)

sentences_that_matter = list(corpus_pos.sentences.keys())
print("Sentences that matter: {}".format(len(sentences_that_matter)))

tokens_changed = set()
tokens_changed_lemma = set()
tokens_changed_deprel = set()
tokens_changed_dephead = set()
tokens_changed_upos = set()
sentences_changed = set()
udpipe_correct = set()
changed_lemmas = {}
all_tokens = 0
all_sentences = 0
cm = set()
for sentid in sentences_that_matter:
    for t, token in enumerate(corpus_pos.sentences[sentid].tokens):
        if not '-' in token.id and token.upos != "PUNCT":
            all_tokens += 1
            try:
                token_pos = corpus_pos.sentences[sentid].tokens[t]
                token_stanza = corpus_stanza.sentences[sentid].tokens[t]
                token_udpipe = corpus_udpipe.sentences[sentid].tokens[t]
            except:
                continue
            important_cols_pos = [token_pos.lemma, token_pos.upos, token_pos.feats, token_pos.dephead, token_pos.deprel]
            important_cols_stanza = [token_stanza.lemma, token_stanza.upos, token_stanza.feats, token_stanza.dephead, token_stanza.deprel]
            important_cols_udpipe = [token_udpipe.lemma, token_udpipe.upos, token_udpipe.feats, token_udpipe.dephead, token_udpipe.deprel]
            if token_stanza.deprel != token_udpipe.deprel:
                cm.add("{}-{}".format(sentid, t))
                if token_udpipe.deprel == token_pos.deprel:
                    udpipe_correct.add("{}-{}".format(sentid, t))
            if important_cols_pos != important_cols_stanza:
                #if token_pos.deprel == "obl:arg" and token_stanza.deprel in ["obj", "iobj"]:
                    #continue
                tokens_changed.add("{}-{}".format(sentid, t))
                if not sentid in sentences_changed:
                    sentences_changed.add(sentid)
                if token_pos.lemma != token_stanza.lemma:
                    tokens_changed_lemma.add("{}-{}".format(sentid, t))
                    if token_pos.lemma.lower() != token_stanza.lemma.lower() and token_pos.upos == "VERB":
                        if not token_pos.lemma.lower() in changed_lemmas:
                            changed_lemmas[token_pos.lemma.lower()] = {}
                        if not token_stanza.lemma.lower() in changed_lemmas[token_pos.lemma.lower()]:
                            changed_lemmas[token_pos.lemma.lower()][token_stanza.lemma.lower()] = 0
                        changed_lemmas[token_pos.lemma.lower()][token_stanza.lemma.lower()] += 1
                if token_pos.upos != token_stanza.upos:
                    tokens_changed_upos.add("{}-{}".format(sentid, t))
                if token_pos.dephead != token_stanza.dephead:
                    tokens_changed_dephead.add("{}-{}".format(sentid, t))
                if token_pos.deprel != token_stanza.deprel:
                    tokens_changed_deprel.add("{}-{}".format(sentid, t))

Already on 'master'


M	Petroles_3/prepare_release.ipynb
Seu ramo está à frente de 'origin/master' por 45 submissões.
  (use "git push" to publish your local commits)


build: 10.53188943862915build: 12.219332456588745build: 11.631117105484009

Sentences that matter: 8955


In [18]:
txt = ""
ocorrencias = 0
c = 1
for lemma in sorted(changed_lemmas):
    ocorrencias += sum([changed_lemmas[lemma][x] for x in changed_lemmas[lemma]])
    txt += "{} ({}) {}".format(", ".join(list(changed_lemmas[lemma].keys())), lemma, "& " if c%4 else "\\\\\n")
    c += 1
print(txt)

print("Lemas corrigidos: {}".format(len(changed_lemmas)))
print("Ocorrências: {}".format(ocorrencias))

aberto (abrir) & absorvar (absorver) & acentuado (acentuar) & adegar (adequar) \\
adsorvir, adsorvar (adsorver) & adver (advir) & aflor (aflorar) & agar, ager (agir) \\
aglomer (aglomerar) & agregado (agregar) & agguardar (aguardar) & abustar (ajustar) \\
alcance (alcançar) & aliado (aliar) & alicerçado (alicerçar) & alinhado (alinhar) \\
alongado (alongar) & aprofundir (aprofundar) & arranjado (arranjar) & assinalado (assinalar) \\
associado, associados (associar) & atinjir (atingir) & avaliado (avaliar) & banhado (banhar) \\
hasear (basear) & bloqueir (bloquear) & bombber (bombear) & quscar (buscar) \\
captura (capturar) & caracterizado (caracterizar) & carrer (carrear) & causa (causar) \\
cedir (ceder) & centralizado (centralizar) & cercado (cercar) & chamado (chamar) \\
cimentado (cimentar) & classificado (classificar) & coleta (coletar) & combinado (combinar) \\
computado (computar) & concentrado (concentrar) & condicionado (condicionar) & conduzido (conduzir) \\
conectado (conect

In [ ]:
# DISPLAY INTERSEÇÕES
print("EXCEPT PUNCT -- Tokens changed: {} / All_tokens: {}".format(len(tokens_changed), all_tokens))
print("Sentences changed: {} / All_sentences: {}".format(len(sentences_changed), len(sentences_that_matter)))
print("CM: {} / Intersection: {} / UDPIPE_CORRECT: {}".format(len(cm), len(tokens_changed.intersection(cm)), len(udpipe_correct)))
print(len(tokens_changed_lemma))
print(len(tokens_changed_upos))
print(len(tokens_changed_dephead))
print(len(tokens_changed_deprel))
print("")
print(len(tokens_changed_lemma.intersection(cm)))
print(len(tokens_changed_upos.intersection(cm)))
print(len(tokens_changed_dephead.intersection(cm)))
print(len(tokens_changed_deprel.intersection(cm)))
print("")
print(len(tokens_changed_lemma.intersection(tokens_changed_upos)))
print(len(tokens_changed_lemma.intersection(tokens_changed_upos).intersection(cm)))
print(len(tokens_changed_lemma.intersection(tokens_changed_deprel)))
print(len(tokens_changed_lemma.intersection(tokens_changed_deprel).intersection(cm)))
print(len(tokens_changed_upos.intersection(tokens_changed_deprel)))
print(len(tokens_changed_upos.intersection(tokens_changed_deprel).intersection(cm)))
print(len(tokens_changed_dephead.intersection(tokens_changed_deprel)))
print(len(tokens_changed_dephead.intersection(tokens_changed_deprel).intersection(cm)))
print(len(tokens_changed_lemma - tokens_changed_upos - tokens_changed_dephead - tokens_changed_deprel))
